In [1]:
!date

Fri Jan  9 02:26:36 PM EST 2026


### Advanced Usage Example for DeepCor fMRI Denoising Toolbox
This example demonstrates comprehensive usage of the deepcor package including:
- Configuration classes (DeepCorConfig, ModelConfig, TrainingConfig, DataConfig)
- Data loading and preprocessing
- Model initialization and training
- Post-training analyses (correlations and contrasts)
- Visualization and tracking
- Both high-level and low-level API usage

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import ants
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
# Import DeepCor configuration classes
from deepcor.config import (
    DeepCorConfig,
    ModelConfig,
    TrainingConfig,
    DataConfig,
)

# Import models
from deepcor.models import CVAE, cVAE, get_model, list_models

# Import training utilities
from deepcor.training import Trainer, save_model, save_brain_signals

# Import data utilities
from deepcor.data import (
    TrainDataset,
    get_roi_and_roni,
    get_obs_noi_list_coords,
    apply_dummy,
    censor_and_interpolate,
    apply_frame_censoring,
    plot_timeseries,
    array_to_brain,
    load_pickle,
    save_pickle,
)

# Import analysis utilities
from deepcor.analysis import (
    run_correlation_analysis_from_spec,
    run_contrast_analysis_from_spec,
    calc_and_save_compcor,
    average_signal_ensemble,
    get_design_matrix,
)

# Import visualization utilities
from deepcor.visualization import (
    init_track,
    update_track,
    show_dashboard,
    save_track,
)

# Import general utilities
from deepcor.utils import safe_mkdir, check_gpu_and_speedup

# Import high-level API
from deepcor.pipeline import DeepCorDenoiser

In [4]:
# ============================================================================
# EXAMPLE 1: GPU Check and Setup
# ============================================================================
print("=" * 80)
print("EXAMPLE 1: GPU Check and Setup")
print("=" * 80)

# Check GPU availability and speedup
result = check_gpu_and_speedup()
print(f"GPU available: {result['gpu_available']}")
if result['gpu_available']:
    print(f"GPU name: {result['gpu_name']}")
    print(f"Average CPU time per op: {result['cpu_time']:.6f} s")
    print(f"Average GPU time per op: {result['gpu_time']:.6f} s")
    print(f"Speedup (CPU/GPU): {result['speedup']:.2f}x")
else:
    print(f"Average CPU time per op: {result['cpu_time']:.6f} s")

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}\n")

EXAMPLE 1: GPU Check and Setup
GPU available: True
GPU name: Tesla V100-SXM2-16GB
Average CPU time per op: 0.005122 s
Average GPU time per op: 0.000190 s
Speedup (CPU/GPU): 26.97x

Using device: cuda:0



In [5]:
# ModelConfig: Configure the model architecture
model_config = ModelConfig(
    latent_dims=(8, 8),  # (shared dim, specific dim)
    beta=0.01,           # KLD loss weight
    gamma=0.0,           # TC loss weight
    delta=0.0,           # RONI zero constraint weight
    scale_MSE_GM=1e3,    # Gray matter reconstruction loss scale
    scale_MSE_CF=1e3,    # Non-gray matter reconstruction loss scale
    scale_MSE_FG=0.0,    # Foreground reconstruction loss scale
    do_disentangle=True  # Enable disentanglement
)
print(f"ModelConfig: latent_dims={model_config.latent_dims}, beta={model_config.beta}")

# TrainingConfig: Configure training parameters
training_config = TrainingConfig(
    n_epochs=100,
    batch_size=1024,
    learning_rate=0.001,
    optimizer='adamw',
    betas=(0.9, 0.999),
    eps=1e-08,
    max_grad_norm=5.0,
    n_repetitions=20  # Number of ensemble repetitions
)
print(f"TrainingConfig: n_epochs={training_config.n_epochs}, batch_size={training_config.batch_size}")

# DataConfig: Configure data preprocessing
data_config = DataConfig(
    n_dummy_scans=0,
    apply_censoring=False,
    censoring_threshold=0.5,
    confound_columns=['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']
)

# Create a complete configuration
config = DeepCorConfig(
    model=model_config,
    training=training_config,
    data=data_config
)

ModelConfig: latent_dims=(8, 8), beta=0.01
TrainingConfig: n_epochs=100, batch_size=1024


In [6]:
# ============================================================================
# EXAMPLE 3: Data Loading and File Paths
# ============================================================================

# Define file paths (customize these for your data)
s = 0
r = 1
sub = 'sub-01'
indir = '../Data/fmriprep-forrest'
analysis_name = 'deepcor-advanced-example'

# Input file paths
epi_fn = os.path.join(
    indir,
    f'{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
)
cf_fn = os.path.join(indir, 'mask_roni.nii')  # Non-gray matter mask
gm_fn = os.path.join(indir, 'mask_roi.nii')   # Gray matter mask
conf_fn = os.path.join(
    indir,
    f'{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_confounds.tsv'
)

print(f"Subject: {sub}")
print(f"Run: {r}")
print(f"EPI file: {epi_fn}")
print(f"Gray matter mask: {gm_fn}")
print(f"Non-gray matter mask: {cf_fn}")
print(f"Confounds file: {conf_fn}")

Subject: sub-01
Run: 1
EPI file: ../Data/fmriprep-forrest/sub-01/ses-localizer/func/sub-01_ses-localizer_task-objectcategories_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
Gray matter mask: ../Data/fmriprep-forrest/mask_roi.nii
Non-gray matter mask: ../Data/fmriprep-forrest/mask_roni.nii
Confounds file: ../Data/fmriprep-forrest/sub-01/ses-localizer/func/sub-01_ses-localizer_task-objectcategories_run-1_bold_confounds.tsv


In [8]:
# Configure correlation and contrast analyses

run_post_analyses = True
if run_post_analyses:
    print("Configuring post-training analyses...")

    # Example: Create design matrix from events file
    print("\n--- Design Matrix ---")
    events_fn = f'../Data/Events/{sub}_ses-localizer_task-objectcategories_run-{r}_events.tsv'
    print(f"Events file: {events_fn}")
    print("Use get_design_matrix(epi, events_fn) to create design matrix")

    # Correlation analyses
    print("\n--- Correlation Analyses ---")
    correlation_analyses = []

    # Example 1: Correlate with face regressor
    correlation_analyses.append({
        'corr_target': None,  # Would be X1['face'].values from design matrix
        'filename': os.path.join(ofdir, f'corr2face_S{s}_R{r}.nii.gz'),
        'plot': True,
        'ROI': f'../Data/Misc/rFFA_final_mask_{sub}_bin.nii.gz'
    })
    print("Added correlation analysis for face regressor")

    # Example 2: Correlate with place regressor (average of house and scene)
    correlation_analyses.append({
        'corr_target': None,  # Would be X1[['house','scene']].values.mean(axis=1)
        'filename': os.path.join(ofdir, f'corr2place_S{s}_R{r}.nii.gz'),
        'plot': True,
        'ROI': f'../Data/Misc/rPPA_final_mask_{sub}_bin.nii.gz'
    })
    print("Added correlation analysis for place regressor")

    # Contrast analyses
    print("\n--- Contrast Analyses ---")
    contrast_analyses = []

    # Example 1: Face contrast
    contrast_analyses.append({
        'contrast_vec': [-1, 5, -1, -1, -1, -1, 0, 0, 0, 0],
        'design_matrix': None,  # Would be X1
        'filename': os.path.join(ofdir, f'contrast_face_S{s}_R{r}.nii.gz'),
        'plot': True,
        'ROI': f'../Data/Misc/rFFA_final_mask_{sub}_bin.nii.gz'
    })
    print("Added contrast analysis for faces")

    # Example 2: Place contrast
    contrast_analyses.append({
        'contrast_vec': [-1, -1, 2, -1, 2, -1, 0, 0, 0, 0],
        'design_matrix': None,  # Would be X1
        'filename': os.path.join(ofdir, f'contrast_place_S{s}_R{r}.nii.gz'),
        'plot': True,
        'ROI': f'../Data/Misc/rPPA_final_mask_{sub}_bin.nii.gz'
    })
    print("Added contrast analysis for places")
else:
    correlation_analyses = []
    contrast_analyses = []

print(f"\nTotal correlation analyses: {len(correlation_analyses)}")
print(f"Total contrast analyses: {len(contrast_analyses)}\n")

Configuring post-training analyses...

--- Design Matrix ---
Events file: ../Data/Events/sub-01_ses-localizer_task-objectcategories_run-1_events.tsv
Use get_design_matrix(epi, events_fn) to create design matrix

--- Correlation Analyses ---


NameError: name 'ofdir' is not defined